In [271]:
import pandas as pd
from random import choices, randint, randrange
import random
import time
import itertools

In [272]:
dataset_courseSchedule = pd.read_csv('../data/CourseSchedule.csv', sep=';')
# sort the dataset_courseSchedule by ET
dataset_courseSchedule = dataset_courseSchedule.sort_values(by=['ET'])

dataset_coursesPP = pd.read_csv('../data/Courses.csv', sep=';')
dataset_professors = pd.read_csv('../data/Professors.csv', sep=';')

semester = 1

dataset_courseSchedule_semester = dataset_courseSchedule[dataset_courseSchedule['ET'] == semester]

timeslots_day = [
    "7:10-8:00",
    "8:00-8:50",
    "8:50-9:40",
    "9:55-10:45",
    "10:45-11:35",
    "11:35-12:25",
    "12:45-13:35",
    "13:35-14:25",
    "14:25-15:15",
    "15:30-16:20",
    "16:20-17:10",
    "17:10-18:00",
    "18:00-18:50",
    "19:00-19:50",
    "19:50-20:40",
    "20:55-21:45",
    "21:45-22:35"]
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

# create a pandas dataframe where you combine the timeslots and days
timeslots = [timeslots_day for i in range(len(days))]

In [273]:
print(dataset_courseSchedule_semester)

  DISCIPLINA  AT  AP  AE  AV                         NOME  CR  SL/B  ET  \
0     HE722C   2   2   0   0     ALGORITMOS E PROGRAMACAO  5W  24/B   1   
1     HE722D   2   2   0   0     ALGORITMOS E PROGRAMACAO  5W  24/B   1   
2     HE822C   4   0   0   0  FUND. DA MATEMATICA p/ ENG.  5W  24/B   1   
3     HF222B   2   0   0   2    FUND. DE ENG. DE SOFTWARE  5W  20/B   1   
4     HE622C   2   2   0   0        LOGICA E CRIATIVIDADE  5W  17/G   1   
5     HE622D   2   2   0   0        LOGICA E CRIATIVIDADE  5W  17/G   1   

   PROFESSOR  ALUNOS MATRIC  
0     11.785             26  
1     11.785             30  
2     11.792             52  
3     13.034             59  
4     13.070             31  
5     13.070             30  


In [274]:
# create a dictionary where all of the courses that a teacher teaches are stored
professor_courses = {}
for professor in dataset_professors['CODE']:
    professor_code = str(professor)
    # the lenght of the professor code has to be 6, if not, add 0's to the right
    while len(professor_code) < 6:
        professor_code += "0"
    professor_courses[professor_code] = []
    for index in range(len(dataset_coursesPP)):
        if dataset_coursesPP['PROFESSOR CODE'][index] == professor:
            professor_courses[professor_code].append(dataset_coursesPP['DISCIPLINA'][index])
print(professor_courses)

{'11.785': ['HE722'], '11.792': ['HE822', 'RG312', 'GM722'], '12.203': ['GB922'], '12.274': ['AC222', 'AD422', 'AD522'], '13.000': ['HF222', 'AC622', 'AD322', 'GK622'], '13.034': ['HE722', 'AB422', 'HF222', 'GL822', 'R0812'], '13.070': ['HE622', 'AD422'], '14.091': ['AB322', 'AE422'], '14.472': ['GJ922', 'R8512', 'AB422', 'GL822', 'R0812'], '14.525': ['HE722', 'AB522', 'AC422', 'AC822'], '14.616': ['AD222'], '14.642': ['GJ922', 'R8512', 'AC922', 'AC922', 'AC122', 'AD522', 'RG712'], '14.776': ['HE722', 'GJ922', 'GK622'], '14.875': ['R8512', 'AD122', 'AC322'], '45.000': ['AC522', 'FU422', 'FV722', 'AB122']}


In [275]:
def generate_bit_length(db_length):
    value = 0
    for i in range(10):
        value += 2**i
        if value >= db_length:
            return i+1

In [276]:
len_classes_encoding = generate_bit_length(len(dataset_courseSchedule_semester))
len_professors_encoding = generate_bit_length(len(dataset_professors))
len_timeslots_day_encoding = generate_bit_length(len(days))
len_timeslots_encoding = generate_bit_length(len(timeslots_day))

print(len_classes_encoding)
print(len_professors_encoding)
print(len_timeslots_day_encoding)
print(len_timeslots_encoding)

3
4
3
5


In [277]:
def generate_genome(length, database):
    binary_value = 100000
    while binary_value >= len(database):
        i = 0
        genome = choices([0,1], k=length)
        binary_value = 0
        reverse_genome = genome[::-1]
        for bit in reverse_genome:
            binary_value += bit*2**i
            i += 1
    return genome

In [278]:
genome_class_binary = generate_genome(len_classes_encoding, dataset_courseSchedule_semester)
genome_professor_binary = generate_genome(len_professors_encoding, dataset_professors)
genome_timeslot_day_binary = generate_genome(len_timeslots_day_encoding, days)
genome_timeslot_binary = generate_genome(len_timeslots_encoding, timeslots_day)

# print all of the genome binarys with string formatting
print("Class: {}".format(genome_class_binary))
print("Professor: {}".format(genome_professor_binary))
print("Timeslot day: {}".format(genome_timeslot_day_binary))
print("Timeslot: {}".format(genome_timeslot_binary))

Class: [0, 0, 0]
Professor: [0, 0, 0, 1]
Timeslot day: [0, 0, 0]
Timeslot: [0, 1, 1, 1, 1]


In [279]:
def translate_genome(genome, database):
    # reverse the genome
    reverse_genome = genome[::-1]
    binary_value = 0
    i = 0
    for bit in reverse_genome:
        binary_value += bit*2**i
        i += 1
    return binary_value

In [280]:
class_part = translate_genome(genome_class_binary, dataset_courseSchedule_semester)
professor_part = translate_genome(genome_professor_binary, dataset_professors)
timeslots_day_part = translate_genome(genome_timeslot_day_binary, days)
timeslots_part = translate_genome(genome_timeslot_binary, timeslots_day)

print("Summary generated genome")
print("Class: {}".format(genome_class_binary))
print("class_binary_value: {}".format(class_part))
print("Class: {}".format(dataset_courseSchedule_semester['DISCIPLINA'][class_part]))
print('\n')

print("Professor: {}".format(genome_professor_binary))
print("professor_binary_value: {}".format(professor_part))
print("Professor: {}".format(dataset_professors['CODE'][professor_part]))
print('\n')


print("Timeslot day: {}".format(genome_timeslot_day_binary))
print("timeslot_day_binary_value: {}".format(timeslots_day_part))
print("Timeslot day: {}".format(days[timeslots_day_part]))
print('\n')

print("Timeslot: {}".format(genome_timeslot_binary))
print("timeslot_binary_value: {}".format(timeslots_part))
print("Timeslot: {}".format(timeslots_day[timeslots_part]))

# write a sentence with the information of the genome
sentence = "The class {} is taught by {} on {} at {}.".format(dataset_courseSchedule_semester['DISCIPLINA'][class_part], dataset_professors['CODE'][professor_part], days[timeslots_day_part], timeslots_day[timeslots_part])
print(sentence)


Summary generated genome
Class: [0, 0, 0]
class_binary_value: 0
Class: HE722C


Professor: [0, 0, 0, 1]
professor_binary_value: 1
Professor: 11.792


Timeslot day: [0, 0, 0]
timeslot_day_binary_value: 0
Timeslot day: Monday


Timeslot: [0, 1, 1, 1, 1]
timeslot_binary_value: 15
Timeslot: 20:55-21:45
The class HE722C is taught by 11.792 on Monday at 20:55-21:45.
